# Introduction
This jupyter notebook gives a brief introduction to groundwater and solute transport modelling with MODFLOW2005 using Visual Studio Code. Visual Studio Code is an open-source integrated development environment (IDE). IDEs are used to develop, run and test software and scripts in various programming languages. This provides an alternative approach to the execution of python scripts in a terminal. All subsequent steps listed in the introductory section can be skipped if you execute this jupyter notebook with binder from your browser. In the FloPy documentation, watch out for binder badge (the symbol of binder). With binder badges you can just open the examples provided by the documentation in your browser and run them interactively, no need for further setup steps.

## 1. Install Python 

To use python, it must be installed on your system. To install it you can download the python installer for version 3.12.4 at: https://www.python.org/ftp/python/3.12.4/python-3.12.4-amd64.exe. If you have arm processors, you need this one: https://www.python.org/ftp/python/3.12.4/python-3.12.4-arm64.exe. Open the installer and make sure to check the add to path option. This is necessary for the system to locate your python executable. For instance, if you type "python --version" in the terminal, you will get an error message, otherwise. As an alternative to double-clicking the installer, you can change the directory using cd in the terminal (opened by searching for cmd):

<code>cd C:Users\<YourUsername>\Downloads
python-3.12.4-amd64.exe
</code>

Afterwards, you can verify that you have pip, the python package manager installed by typing in the terminal:

<code>
pip --version
</code>

The terminal should yield an output similar to this: "pip 24.0 from C:\Users\cgaertner\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip (python 3.12)"

## 2. Install VSCode
 VS Code can be installed on Windows 10 and 11 via Microsoft Store. The Microsoft Store can be found in the windows search bar. Make sure to pick the blue version (not the purple one). In VSCode, you can find the extensions on the left side bar. Search for python and Jupyter and install both extensions. This document is a jupyter notebook (.ipynb). Jupyter notebooks provide a convenient means to combine code cells and so called markdown cells for descriptive purposes and are often used for educational purposes. Each code cell can be run individually allowing for a separate inspection of intermediary code results. Make sure to restart VS Code once you have installed the extensions.

## 3. Create and activate a virtual environment (.venv)
There are two ways to use pyton interpreters. The first one is to use the global interpreter that you just installed on your system via the comand line interface (cmd). You can then install python packages globally by running ```pip install <package-name>```. However, this is often a bad practice. Dependencies are the packages used by a script or programm. Because of compatibility issues, often certain programm versions depend on each other, e. g. package A relying on package B to be given at below or above a specified version. Moreover, the creation of virtual environments allows to clearly separate the distinct dependencies and versions required by your code.

First, you need to create a folder that will contain your virtual environment. Create a projects folder under the .vscode directory (C:\Users\<YourUsername>\.vscode\Projects). In the projects folder create a new folder called <SoluteTransportExercise>. To create a virtual environment, click on "Terminal" in VS Code in the top bar and open a new terminal. Change to the directory where you want to create the venv, i. e. type:

<code>
cd C:\Users\<YourUserName>\.vscode\Projects\SoluteTransportExercise.
</code>

If you are already in the Projects directory you can just type "" cd SoluteTransportExercise to navigate to the target folder. Then type: 

<code>
python -m venv .venv
</code>

Now you are informed by VS Code that a new venv folder has been created successfully. Confirm the question if you want to add it to your workspace folder.
You should now see a .venv folder being created in the SoluteTranpsortExercise folder. You can see the folders by clicking on the explorer icon in the topmost left sidebar of VS Code. The next step is to activate the virtual environment. In your venv folder, three is a Scripts folder containing a script called <activate>. To activate the environment, type the following command in the script: 

<code>
C:\Users\<YourUsername>\.vscode\Projects\SoluteTransportExercise\.venv\Scripts\activate 
C:\Users\cgaertner\.vscode\Projects\SoluteTransportExercise\.venv\Scripts\activate 
</code>

You should see a (.venv) at the left side of the terminal, indicating that the virtual environment has been successfully activated. Now you can install packages into the environment using pip. Packages installed with pip will typically be stored in your venv folder in the Lib\site-packages folder. If your activation was not successfull, e. g. because of execution policies, type the following commands in the terminal and retry the activation process: 

<code>
Get-ExecutionPolicy
Set-ExecutionPolicy RemoteSigned -Scope CurrentUser
C:\Users\<YourUsername>\.vscode\Projects\SoluteTransportExercise\.venv\Scripts\activate 
C:\Users\cgaertner\.vscode\Projects\SoluteTransportExercise\.venv\Scripts\activate 
</code>

Note that the (.venv) might only be visisble if your select the terminal to be a command prompt (cmd) on the plus sign with the dropdown menu to the right of the terminal window. If you want to install packages with pip, your environment must be active.

## 4. Install the required packages: 
Open a terminal in your virtual environment and run:

<code>
pip install flopy numpy matplotlib
</code>

From the terminal output you will recognize that all other dependencies of these listed packages are installed automatically under the hood.

## 5. Selecting your venv as a jupyter notebook kernel
Unfortunately, using jupyter notebooks within virtual environments requires additional efforts. This comprises creating a jupyter kernel and selecting the python interpreter of the venv to access the corresponding packages. To create a jupyter kernel, make sure your venv is active and run the following commands:

<code>
pip install ipykernel notebook
python -m ipykernel install --user --name=venv --display-name "Python (venv)"
</code>

This installs an ipython kernel associated with the virtual environment for the current user. --name==venv assumes that you have named your virtual environment venv as suggested in this tutorial. Now you have to select the created ipython kernel. Go to the top menu of VS Code and click on "View". Open the command palette (alternatively: Ctrl+Shift+P) and search for "Notebook: Select Notebook Kernel". In the top right corner of your jupyter notebook you should now see the name of the newly created kernel.



## Moflow and FloPy
MODFLOW is the U.S. Geological Survey modular finite-difference flow model, which is a hydrogeological code that has been used to solve the groundwater flow equation and related problems since 1988. Modflow is considered as international state-of-the-art for simulating and predicting groundwater flow and interactions between the groundwater and surface water. It is characterised by its modular structure, meaning that packages for e. g. initial conditions, wells, boundary conditions, recharge, discretization and layer or node flow properties have to be added separately. More detailed information can be found under https://www.usgs.gov/mission-areas/water-resources/science/modflow-and-related-programs. 

The first part of this jupyter notebook deals with setting up the groundwater flow model to calculate the flow field (i. e. the hydraulic heads and the resulting groundwater flow velocities in each cell). On the basis of the simulated flow field, solute transport is calculated assuming a decoupling between groundwater flow and contaminant spread. This decoupling implies that changes in the flow field due to concentration changes are marginal and do not affect the fluid density. To simulate the contaminant transport, the MT3DMS (Modular Transport, 3-Dimensional, Multi-Species model) transport code is utilized. Note that the latest version of Modflow (MODFLOW 6) does not support for MT3DMS yet. On the other hand, the local grid refinement utility which may be used to locally increase the spatial discretization around wells exhibiting increased hydrualic gradients by nature is currently only available for Modflow 6.

FloPy is a python package that allows to create, run and automate MODLFOW Models and modelling workflows by using python scripts. The documentation can be found here: https://flopy.readthedocs.io/en/latest/introduction.html. FloPy can be intalled with pip:

<code>
pip install flopy numpy matplotlib
</code>

The next step is to get the executables (.exe) of the MODFLOW and MT3DMS code. FloPy includes a get-modflow utility to install USGS MODFLOW and related programs for Windows, Mac or Linux. If FloPy is installed, the utility is available in the Python environment as a get-modflow command. The script flopy/utils/get_modflow.py has no dependencies and can be invoked independently. Open a terminal and type the absolute path of the script followed by the absolute path of the Scripts directory of your venv as the "bindir" command line argument. Since the scripts folder is already in the path environment variable, it is suitable to located the executables here. Note: Once a binary (an executable) is in the path, you do not need to specify the full path but only the filename to run it.

<code>
C:\Users\<YourUsername>\.vscode\Projects\SoluteTransportExercise\.venv\Scripts\get-modflow.exe C:\Users\<YourUsername>\.vscode\Projects\SoluteTransportExercise\.venv\Scripts
C:\Users\cgaertner\.vscode\Projects\SoluteTransportExercise\.venv\Scripts\get-modflow.exe C:\Users\cgaertner\.vscode\Projects\SoluteTransportExercise\.venv\Scripts
</code>


## Conventions
For the numbering of rows, columns and layers (i, j, k) of the discretization grid, it is important to mention that numbers start at zero in python. Water flowing into the model is considered positive, water leaving the domain counts negative. By default all boundaries in MODFLOW are no-flow boundaries, so you have to specify the boundary conditions explicitly cell- or nodewise. By default, Modflow uses days as time periods and values such as hydraulic conductivity must be adjusted accordingly.

## Model Description
The model comprises a layered system of horizontal aquifers with an areal extent of 5 x 5 km². The hydraulic conductivity is assumed to be isotropic (uniform in all directions). All layers have a porosity of 0.3. The second layer acts as an aquitard (0.01 m/d), separating the highly conductive top layer (20 m/d) and the two lower aquifers (5 m/d and 10 m/d). The recharge rate of 0.0005 m/d is uniformly assumed across the model domain. The right boundary of the investigation area is formed by a river that can be represented by a Dirichlet boundary condition (58 m hydraulic head). 

The first step is import all required modules at the beginning of your python script. The python interpreter reads from the top to the bottom of a script implying these modules have to be initialized at the beginning in order to be recognized. Some modules are imported using aliasing denominated by followed by a shorthand designation. This is not obligatory but usually a convenient method to keep the code short. For instance, numpy is convnetionally abbreviated as np.

In [2]:
import os
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("grayscale")
plt.rcParams['figure.figsize'] = (8, 6) # default figure size
plt.rcParams['figure.autolayout'] = True # same as tight layout
import flopy

After the module imports, the names of the used executables and a workspace directory in which the results are outputted are defined as variables. Note that since we already have the executables in the path, only their filenames need to be specified. Based on the operating system (Windows in VS Code or Linux with Binder) executables may have extenions (.exe) or not.

In [3]:
model_ws = "./mt3d_test"
model_name = "aquifer"
if os.name == 'nt':
    print("The operating system ist Windows")
    exe_name_mf = 'mf2005.exe'
    exe_name_mt = 'mt3dms.exe'

elif os.name == 'posix':
    print("the operating system is Linux or MacOS")
    exe_name_mf = 'mf2005'
    exe_name_mt = 'mt3dms'
    os.chmod('mf2005', 0o755) # grant rights to the executable
    os.chmod('mt3dms', 0o755) # grant rights to the executbale
else:
    print("Unknown operating system")

The operating system ist Windows


Now we must define some basic parameters of the model for creating instances of the corresponding Moflow packages. The variable names are a bit cryptic since they already follow the naming conventions used by Modflow.

In [ ]:
# Define the grid properties
L_x = 5000  # meters
L_y = 5000  # meters
ncol = 100  # number of columns
nrow = 100 # number of rows
delc = L_x / ncol  # column width in meters (i.e., 50 m), del stands for delta
delr = L_y / nrow  # row width in meters (i.e., 50 m), del stands for delta

# Layers
nlay = 4
top = 60
botm = [40, 35, 15, 0]  # bottom of each layer

# Aquifer properties
k_hor = [20, 0.01, 5, 10]  # horizontal hydraulic conductivity in m/d for each layer bottom to top
k_ver = [20, 0.01, 5, 10]  # vertical hydraulic conductivity in m/d for each layer bottom to top
porosity = 0.3  # porosity of all layers

# Boundary conditions
head_river = 58  # hydraulic head in the river in meters
Q_well = -3000  # pumping rate in m^3/d, discharge is negative
recharge = 0.0005  # recharge rate in m/d

# Groundwater Model (MODFLOW)
Now we can create the Modflow groundwater model using this code fragment. It specifies the name of our model ("Aquifer"), the name of our modflow executable (version "mf2005") and the model workspace where our input and output files will be generated ("mt3d_test"). The last "Modflow" is written with a capital letter, indicating that we are creating an instance of a class. Pro Tip: By hovering over each element, attribute or keyword argument in your IDE you can see the required data type (e. g. int, float, bool) and purpose in the documentation popping up. This is a standard feature in most IDEs and means you do not need to learn each modules methods by heart but rather learn how to read the docs.

In [ ]:
mf = flopy.modflow.Modflow(model_name, exe_name=exe_name_mf, model_ws=model_ws)

## Adding the discretization package (ModlowDis)
Numerical (approximate) solutions of partial differential equations such as the groundwater flow and solute transport equations are often solved using the finite difference method (FDM). This requires the spatial discretization of the model domain into a structured grid of rectangular cells. Horizontally for each row (i) and column (j) and vertically for each layer (j), a modell cell is uniquely identified by a tuple of the coordinate indices (i, j, k). Apart from the spatial discretization, a temporal discretization is required. In Modflows terminology, the total simulated time is divided into stress periods (spd). Stress periods occur, once some boundary conditions change in the model. By default the total simulated time is represented by one stress period. Each stress period can have a number of time steps. If more than one stress periods occurr, the parameters for these stress periods are passed as lists. E. g. in the perlen argument, the first variable characterizes the length of the first stress period in which contaminant is injected into the third aquifer layer. Afterwards, the injection ceases and a period of 200 years is considered. This period is divided into the corrsponding number of intermediary time steps specified at the second index (1) at nstp. Not that the choice of the discretizaztion has significant impact on the accuracy of the solution, the numerical stability and the required computational time.

In [ ]:
# Discretization package 
dt_spd1 = 365 # days
dt_spd2 = 7300 # days
perlen = [dt_spd1, dt_spd2]  # Length of each stress period in days - 200 years
nstp = [2, 40]  # Number of time steps in each stress period
steady = [True, True] # The hydraulic heads through the well extractions are already in equilibrium
dis = flopy.modflow.ModflowDis(mf, nlay=nlay, nrow=nrow, ncol=ncol, delr=delr, delc=delc, top=top, botm=botm,
                               nper=2, perlen=perlen, nstp=nstp, steady=steady)

## Adding the basic package for Modflow (ModflowBas)
The basics package for Modflow is used to define cell type, boundary conditions and initial conditions.

The `ibound` line creates a 3D numpy array named ibound with dimensions `(nlay, nrow, ncol)`, representing the number of layers `(nlay)`, rows (`nrow`), and columns (`ncol`) in the model grid. The `np.ones` function of numpy initializes all elements of the array to 1, indicating that all cells are active (i.e., part of the model domain where groundwater flow is actively simulated and not predetermined).

Each cell can be either:
- active: variable head (>0)
- inactive: outside the domain (0)
- constant head: boundary (<1)

The `strt` parameter (starting heads) represents an array created with the same dimensions `(nlay, nrow, ncol)` and first initialised with ones and then multiplied by the intial head (concept of broadcasting numpy arrays which affects every elemtn of an array). Since the initial hydraulic head is not known everywhere in the domain of interest, it is at first estimated to be equal to the hydraulic head of the river everywere in the domain. For steady-steate problems, the actual initial distribution of hydraulic heads is iteratively calculated from the first guess distribution by respecting the mass balance. Even though the calculated stationary initial distribution of heads does not depend on the initial guess but only on the boundaries, more realistic estimates result in faster convergence. Unlike for steady-state problems, the initial head distribution has a significant impact on the transient evolution of hydraulic heads. One strategy could be the interpolation of the piezometric heads between different wells to increase physical realism.

In [ ]:
# Basic package
ibound = np.ones((nlay, nrow, ncol), dtype=np.int32)
strt = np.ones((nlay, nrow, ncol), dtype=np.float32) * head_river
bas = flopy.modflow.ModflowBas(mf, ibound=ibound, strt=strt)

## Adding the layer property flow package (ModflowLpf)
This package is used to define hydraulics properties such as horizontal and vertical hydraulic conductivity for each layer. `mf` is our modflow model that we created in the beginning. The `laytyp` list uses a binary flag to indicate whether a layer is confined (0) or convertible (1), i. e. behaves like an unconfined aquifer but can become confined depending on the head level. `hk` and `vka`are lists of the horizontal and vertical hydraulic conductivities defined earlier in the script. The `sy` parameter denominates the specific yield relevant in the unconfined top aquifer. The specific yield is the part of porosity that is freely availbale for groundwater filtration and not retented by capillary forces. The total porosity is formed by the sum of the specific yield and the specific retention due to capillary forces. Setting the specific yield and the porosity equal is a simplification due to a lack of data.

In [ ]:
lpf = flopy.modflow.ModflowLpf(mf, laytyp=[1, 0, 0, 0], hk=k_hor, vka=k_ver, ipakcb=53, sy=porosity)

## Adding the recharge package (ModflowRch)
This adds a constant recharge source uniformly across all model cells to upper aquifer. The recharge is assumed to be constant for both stress periods. 

In [ ]:
# Recharge package
rch = flopy.modflow.ModflowRch(mf, rech={0: recharge, 1: recharge})

## Adding the well package (ModflowWel)
The well package is used to model the impact of wells with specified injection or extraction rates on the piezometric head surface. The stress period data `wel_spd` are a dictionary with the value being a list of lists for each well. These list follow the `layer, row, column, flux` syntax. The groundwater is extracted from the third layer (2). The wells are vertically oriented at 50 (cell size) x 20, 25, 30 = 1000, 1250 and 1500 m. Note its a good practice to avoid hardcoding values and using relative positions, here and they are rounded by an integer to avoid errors.

In [ ]:
# Well package
wel_spd = {
    0: [[2, int(0.4 * nrow), int(0.9 * ncol), Q_well], [2, int(0.5 * nrow), int(0.9 * ncol), Q_well], [2, int(0.6 * nrow), int(0.9 * ncol), Q_well]],
    1: [[2, int(0.4 * nrow), int(0.9 * ncol), Q_well], [2, int(0.5 * nrow), int(0.9 * ncol), Q_well], [2, int(0.6 * nrow), int(0.9 * ncol), Q_well]]
}

wel = flopy.modflow.ModflowWel(mf, stress_period_data=wel_spd)


## Adding the constant head package (ModflowChd)
The constant head package is added to the Modflow model. You might be wondering why we need this package since we already defined the cell types within the basic Modflow class (ModflowBas). However, its is required to instruct Modflow to keep boundary conditions for the specified cells over all (2) stress periods constant. The river is located vertically at the right boundary, so the for loop aggregates in the last column (`n_col - 1`) all model cells. The stress period data is then again passed as a dictionary.

In [ ]:
# Specified-Head Package (CHD)
chd_spd = []
for row in range(nrow):
    chd_spd.append([0, row, ncol-1, head_river, head_river])
chd = flopy.modflow.ModflowChd(mf, stress_period_data={0: chd_spd, 1: chd_spd})

## Adding the output controll package (ModflowOC)
This package specifies the data and their timesteps to write simulation outputs. The `stress_period_data` argument accepts a dictionary with the keys being a tuple of the form `(iper, istp)` with `iper` being the stress period and `istp` being the timestep.

In [ ]:
# Output Control package
oc = flopy.modflow.ModflowOc(mf, stress_period_data={(0, 0): ['save head', 'save budget'],
                                                     (1, nstp[1]): ['save head', 'save budget']},
                             compact=True)

## Adding the Preconditioned Conjugate-Gradient (PCG) solver and Link-Mt3DMS package
In this part of the code, we set up the solver package and the Link-MT3D package for our MODFLOW model.

First, we configure the Preconditioned Conjugate Gradient (PCG) solver package using the ModflowPcg class from FloPy. The solver is essential for solving the system of linear equations that arises from the discretization of the groundwater flow equations. The hclose parameter is set to 1e-5, which defines the head change criterion for convergence. This means that the iterative solution process will stop when the maximum head change between iterations is less than 1e-5 units. The rclose parameter, set to 1e-3, defines the residual criterion for convergence, indicating that the solution process will stop when the maximum residual (the difference between the left and right sides of the equation) is less than 1e-3 units. We also specify the maximum number of outer iterations (mxiter) as 50 and the maximum number of inner iterations per outer iteration (iter1) as 30. These settings control the iterative process and help ensure that the solver converges efficiently.

Next, we set up the Link-MT3D package using the ModflowLmt class from FloPy. This package is crucial for linking the MODFLOW model to the MT3DMS transport model, enabling the simulation of contaminant transport within the groundwater flow model. We define the output file name for the flow-transport link file using the output_file_name parameter, which is set to f"{model_name}.ftl". This file will contain the flow information required by the MT3DMS model. Additionally, we specify the output file unit as 54, the file header format as 'extended', and the file format as 'unformatted'. These settings determine how the flow-transport link data is written to the file and ensure that it is correctly interpreted by the MT3DMS model during the transport simulation.

By setting up the PCG solver and the Link-MT3D package, we ensure that our MODFLOW model is well-prepared for both efficient flow simulation and subsequent contaminant transport modeling.

In [ ]:
# Solver package (PCG for MODFLOW 2005) and 
pcg = flopy.modflow.ModflowPcg(mf, hclose=1e-5, rclose=1e-3, mxiter=50, iter1=30)

lmt = flopy.modflow.ModflowLmt(mf, output_file_name="mt3d_link.ftl",
                               output_file_unit=54, output_file_header='extended',
                               output_file_format='unformatted')

## Write input files and run groundwater simulation

In [ ]:
# Write MODFLOW input files
mf.write_input()

# Run MODFLOW model
mf.run_model()

## Visualizing and postprocessing of intermediate results
In this section, we visualize the hydraulic head results obtained from the MODFLOW simulation. 

We start by accessing the head data from the output file using the HeadFile class from FloPy utilities. The file containing the head data is located in the model workspace, and we use the filename format specified earlier. The get_data method is then used to extract the head data for a specific stress period and time step, which in this case is the first time step of the first stress period (kstpkper=(0, 0)). We also print the type of the head variable to confirm that we have successfully retrieved the data.

Next, we calculate and print the maximum and minimum hydraulic head values in the model to get an overview of the range of hydraulic heads simulated. This provides a quick check on the simulation results, helping us identify any potential issues or anomalies.

To visualize the spatial distribution of the hydraulic head, we use the PlotMapView class from FloPy. This class provides a convenient way to create map views of the model results. We plot the hydraulic head data for the second model layer (index 1) using the plot_array method with a colormap (cmap) of 'jet', which provides a clear gradient of colors representing different head values. We also add a color bar to the plot to indicate the range of head values, with the label "Head (m)". Matplotlib is required as a dependency and colormap names are acording to matplotlib.

Additionally, we create contour lines on the plot to represent different head levels using the `contour_array`method. The contour levels are set to range from 50 to 70 meters with blue lines (colors="b") and a line width of 0.5. We then label the contour lines with their corresponding head values using the clabel method, formatting the labels to display as whole numbers.

Finally, we set the title of the plot to "Hydraulic heads in layer 2" and display the plot using plt.show(). This visualization helps us understand the distribution of hydraulic heads within the model layer, providing valuable insights into the groundwater flow patterns simulated by the model.

In [ ]:
# Visualize head results
hds = flopy.utils.HeadFile(os.path.join(model_ws, f"{model_name}.hds"))
head = hds.get_data(kstpkper=(0, 0))
print(type(head)) # this is useful to find out what kind of object you are dealing with in python
print('Maximum head in the model:', head.max())
print('Minimum head in the model:', head.min())

pmv = flopy.plot.PlotMapView(model=mf)
gm = pmv.plot_array(head[2], cmap="jet")
plt.colorbar(gm, shrink=0.7, label="Head (m)")
cs = pmv.contour_array(head[2], levels=range(50, 70), colors="b", linewidths=0.7)
plt.clabel(cs, fmt='%1.0f')
plt.title("Hydraulic heads in layer 2")
plt.show()

# Solute Transport Model (MT3DMS)
In this section we define the modelname and define the modflow model and the model workspace where the outputfiles can be found. Note that after succesfully running the groundwater simulation, the `mt3d_test`directory should be populated with a variety of input files for each of the modules and the outputted `.hds` file. With the modelname we define an instance of the Mt3Dms class which is required to model solute transport and is only comptaible with Modflow 2005.

In [ ]:
# Define MT3DMS model
mt3dname = model_name + "_mt3d"
mt = flopy.mt3d.Mt3dms(modelname=mt3dname, exe_name=exe_name_mt, modflowmodel=mf, model_ws=model_ws)

## Adding the basic transport package (Mt3dBtn)
In this section, we configure the basic transport package for the MT3DMS model. The Basic Transport (BTN) package is crucial for defining the fundamental settings and initial conditions for the solute transport simulation.

First, we initialize the `icbund` array using the `np.ones` function. This array defines the boundary conditions for solute transport, where a value of 1 indicates active cells that participate in the transport process. The dimensions of the `icbund` array are set to match the model grid, with layers (`nlay`), rows (`nrow`), and columns (`ncol`).

Next, we initialize the `sconc` array using the `np.zeros` function. This array sets the initial concentration of the solute in the groundwater model. Here, we set all initial concentrations to zero, indicating that there is no initial solute present in the system. The dimensions of the `sconc` array are also set to match the model grid.

We then create the `btn` object using the `Mt3dBtn` class from FloPy's MT3DMS module. This object represents the Basic Transport package and includes several important parameters:

- `icbund` is passed to define the boundary conditions for the transport model.
- `prsity` is set to the porosity of the aquifer material, which was defined earlier.
- `sconc` is set to 0, indicating that the initial solute concentration throughout the model domain is zero.
- `nper` is set to 2, indicating that there are two stress periods in the simulation.
- `perlen` is set to the previously defined stress period lengths.
- `nstp` is set to the previously defined number of time steps per stress period.
- `dt0` is set to 0.1, defining the initial transport time step length.
- `obs` is a list of observation points where solute concentrations will be monitored during the simulation. In this case, we specify two observation points, one of them corresponding to the center of the middle well.
- `timprs` is a list of time points at which the concentrations will be saved. This list is generated using np.arange to create time points from 0 to 73000 days in steps of 100 days.
By setting up the Basic Transport package with these parameters, we ensure that the MT3DMS model is properly initialized for the solute transport simulation, with appropriate initial conditions, boundary conditions, and observation points.

In [ ]:
# Basic transport package
icbund = np.ones((nlay, nrow, ncol), dtype=np.int32)
sconc = np.zeros((nlay, nrow, ncol), dtype=np.float32)
btn = flopy.mt3d.Mt3dBtn(mt, icbund=icbund, prsity=porosity, sconc=0, nper=2, perlen=perlen, nstp=nstp,
                         dt0=0.1, obs=[(2, int(0.4 * nrow), int(0.9 * ncol)), (2, int(0.5 * nrow), int(0.9 * ncol))],
                         timprs=list(np.arange(0, dt_spd2, 100)))

## Adding the Source-Sink Mixing package (Mt3dSsm)

In this section, we define a new contaminant source for the MT3DMS model and configure the Source/Sink Mixing (SSM) package. The contaminant source will be active only during the first stress period and will be deactivated in the subsequent stress period.

First, we define the contaminant source with its specific location and concentration. The contaminant_source variable is a list containing the layer, row, column, and concentration of the contaminant source. In this example, the contaminant source is located in layer 1, row 70, column 50, with a concentration of 1000.0 mg/L.

Next, we obtain the dictionary of source/sink types using the itype_dict method from the Mt3dSsm class. This dictionary maps the different types of sources and sinks to their corresponding integer codes used in the MT3DMS model.

For the first stress period, we create a list, cwell_info_1, to store the contaminant source information. We loop through the contaminant_source list and add the details to the cwell_info_1 list. This ensures that the specified concentration of the contaminant source is accounted for during the first stress period using the itype['CC'] for constant concentration.

Since the contaminant source is only active in the first stress period, we create an empty list, cwell_info_2, for the second stress period, indicating that there are no active sources or sinks in this period.

We combine these lists into a dictionary, spd_mt, which maps each stress period to its respective source/sink data. The Mt3dSsm object, ssm, is then created using this dictionary. This object represents the Source/Sink Mixing package and integrates the source information into the MT3DMS transport model.

By setting up the contaminant source to be active only in the first stress period and configuring the SSM package without wells, we ensure that the MT3DMS model accurately simulates the transport of solutes from the specified source during the initial period and correctly handles the absence of sources in subsequent periods.

In [ ]:
# Define a new contaminant source
contaminant_source = [[2, int(0.72 * nrow), int(0.72 * ncol), 1000.0]]  # [layer, row, column, concentration]

# Source/Sink Mixing package
itype = flopy.mt3d.Mt3dSsm.itype_dict()

# First stress period with contamination source
cwell_info_1 = []
for source in contaminant_source:
    layer, row, col, conc = source
    cwell_info_1.append([layer, row, col, conc, itype['CC']])  # Type -1 for constant concentration

# Second stress period with the same cell but zero concentration
cwell_info_2 = []
for source in contaminant_source:
    layer, row, col, conc = source
    cwell_info_2.append([layer, row, col, 0.0, itype['CC']])  # Type -1 for constant concentration with zero value

spd_mt = {0: cwell_info_1, 1: cwell_info_2}
ssm = flopy.mt3d.Mt3dSsm(mt, stress_period_data=spd_mt)


## Adding advection and dispersion packages (Mt3d)
In this section, we configure the advection and dispersion packages for the MT3DMS model, which are essential for simulating the transport processes of solutes in groundwater.

**Advection Package**
Advection is the process by which solutes are transported by the bulk movement of flowing groundwater. It is driven by the velocity field within the aquifer, which in turn is influenced by hydraulic gradients and the permeability of the geological medium. The advection package is configured using the Mt3dAdv class from FloPy.

`mixelm` is an integer flag for the advection solution option. 

- = 0, the standard finite-difference method with upstream or central-in-space weighting, depending on the value of NADVFD; 
- = 1, the forward-tracking method of characteristics (MOC);
- = 2, the backward-tracking modified method of characteristics (MMOC);
- = 3, the hybrid method of characteristics (HMOC) with MOC or MMOC automatically and dynamically selected;
- = -1, the third-order TVD scheme (ULTIMATE).

The `mixelm` parameter is set to -1, which specifies the use of the third-order TVD (Total Variation Diminishing) scheme for solving the advection term. This scheme is known for its accuracy and stability in handling sharp concentration fronts and minimizing numerical dispersion.

**Dispersion Package**
Dispersion in groundwater refers to the spreading of solute particles due to the combined effects of mechanical dispersion and molecular diffusion. Mechanical dispersion is caused by variations in velocity within the porous medium, while molecular diffusion is the random movement of solute particles at the molecular level.
Dispersion tends to smear the concentrations.

The dispersion package is configured using the `Mt3dDsp` class from FloPy, with several key parameters:

- `al` (longitudinal dispersivity): This parameter represents the spreading of the solute in the direction of groundwater flow. It is set to 10.0 meters.
- `trpt` (ratio of horizontal transverse dispersivity to longitudinal dispersivity): This ratio indicates how much solute spreads perpendicular to the flow direction in the horizontal plane. It is set to 0.2, meaning horizontal transverse dispersivity is 20% of the longitudinal dispersivity.
- `trpv` (ratio of vertical transverse dispersivity to longitudinal dispersivity): This ratio indicates how much solute spreads perpendicular to the flow direction in the vertical plane. It is set to 0.01, meaning vertical transverse dispersivity is 1% of the longitudinal dispersivity.
- `dmcoef` (effective molecular diffusion coefficient): This coefficient represents the diffusion of solute particles due to molecular motion and is set to 1e-4 square meters per day.

By configuring the advection and dispersion packages, we ensure that the MT3DMS model accurately simulates the transport of solutes through groundwater, accounting for the processes of bulk movement and spreading due to both mechanical dispersion and molecular diffusion.

In [ ]:
# Advection package
adv = flopy.mt3d.Mt3dAdv(mt, mixelm=3)

# Dispersion package
al = 10.0  # longitudinal dispersivity
trpt = 0.1  # ratio of horizontal transverse dispersivity to longitudinal dispersivity
trpv = 0.01  # ratio of vertical transverse dispersivity to longitudinal dispersivity
dmcoef = 1e-5  # effective molecular diffusion coefficient
dsp = flopy.mt3d.Mt3dDsp(mt, al=al, trpt=trpt, trpv=trpv, dmcoef=dmcoef)

## Adding the Generalized Conjugate Gradient (Mt3dGcg) solver package
In this section, we configure the Generalized Conjugate Gradient (GCG) solver package for the MT3DMS model. The solver package is crucial for efficiently solving the system of linear equations that arise during the simulation of solute transport processes.

The GCG solver is configured using the `Mt3dGcg`class from FloPy. This solver is particularly well-suited for handling the large, sparse matrices that are typical in groundwater modeling, providing both accuracy and computational efficiency.

- `mxiter` (maximum outer iterations): This parameter specifies the maximum number of outer iterations allowed for the solver. In this case, it is set to 1, indicating that the solver will perform a single outer iteration.
- `iter1` (maximum inner iterations): This parameter defines the maximum number of inner iterations allowed per outer iteration. It is set to 50, allowing the solver to iterate up to 50 times to converge within each outer iteration.

The solver package plays a critical role in ensuring that the transport simulation runs efficiently and accurately. The outer iterations help in handling non-linearities in the transport equations, while the inner iterations focus on solving the linear system within each outer iteration. By balancing the number of outer and inner iterations, the solver can achieve convergence within a reasonable computational effort.

The configuration of the GCG solver in this example ensures that the MT3DMS model can handle the transport simulation effectively, providing reliable results for the distribution and movement of solutes in the groundwater system.

In [ ]:
# GCG solver package
gcg = flopy.mt3d.Mt3dGcg(mt, mxiter=1, iter1=50)

## Write input files and run solute transport simulations

In [ ]:
# Write the MT3DMS input files
mt.write_input()

# Run the MT3DMS model
mt.run_model()

## Visualization of the flow field and concentration in the third aquifer
In this section, we visualize the transport results obtained from the MT3DMS simulation. This involves examining the solute concentration distribution and the groundwater flow field.

### Concentration data
First, we access the concentration data from the output file using the UcnFile class from FloPy utilities. The file containing the concentration data is located in the model workspace, and we use the filename 'MT3D001.UCN'. We then use the get_times method to retrieve the available simulation times, which gives us an overview of the time steps for which concentration data is available. We print these times for reference.

Next, we visualize the solute concentration distribution. Again, using the PlotMapView class, we plot the concentration data for the third model layer (index 2) using the plot_array method with a colormap (cmap) of 'jet', which provides a clear gradient of colors representing different concentration levels. We also add a color bar to the plot to indicate the range of concentration values, with the label "Concentration (mg/L)". We set the title of the plot to "Contaminant concentration in layer 2" and display the plot using plt.show().

Next, we extract the concentration data for the last time step using the get_data method with the totim parameter set to the last available time. We then print the maximum and minimum concentration values in the model to get a quick overview of the range of concentrations simulated. Furthermore, we extract the breakthrough curve in the middle well.

In [ ]:
# Visualize transport results
ucn_filepath = os.path.join(model_ws, 'MT3D001.UCN')

# Visualize transport results
ucnobj = flopy.utils.UcnFile(ucn_filepath)
times = ucnobj.get_times()
print(f"Available times: {times}")
last_time = times[-1]
concentration = ucnobj.get_data(totim=last_time)
print('Maximum concentration in the model:', concentration.max())
print('Minimum concentration in the model:', concentration.min())

# Plot the concentration in the third layer (layer index 2)
pmv = flopy.plot.PlotMapView(model=mf, layer=2)
cm = pmv.plot_array(concentration[2], cmap="jet")
plt.colorbar(cm, shrink=0.7, label="Concentration (mg/L)")
plt.title("Contaminant concentration in layer 3")
plt.show()

# Define the observation point (middle well)
obs_layer = 2
obs_row = int(0.5 * nrow)
obs_col = int(0.9 * ncol)

# Plot the breaktrough curve over time at the middle well
concentration_time_series = []
for time in times:
    concentration = ucnobj.get_data(totim=time)
    concentration_time_series.append(concentration[obs_layer, obs_row, obs_col])

# Plot the breakthrough curve
plt.figure(figsize=(10, 6))
plt.plot(times, concentration_time_series, marker='o', linestyle='-')
plt.xlabel('Time (days)')
plt.ylabel('Concentration (mg/L)')
plt.title('Breakthrough Curve at the Middle Well')
plt.grid(True)
plt.show()

### Cell budgets and flow field
After the execution of the MODFLOW model, we also access the flow field data using the CellBudgetFile class from FloPy utilities. The file containing the flow data is located in the model workspace, and we use the filename format specified earlier. We extract the flow data for the right face (FLOW RIGHT FACE) and front face (FLOW FRONT FACE) using the get_data method.

To visualize the groundwater flow field, we use the PlotMapView class from FloPy. This class provides a convenient way to create map views of the model results. We plot the flow vectors for the third model layer (index 2) using the plot_vector method. This helps us understand the direction and magnitude of groundwater flow within the model layer. We set the title of the plot to "Groundwater flow field in layer 2" and display the plot using plt.show().

By visualizing both the flow field and the concentration distribution, we can analyze the transport processes within the groundwater system, gaining insights into how contaminants move and spread over time.

In [ ]:
# After the execution of the MODFLOW model
cbb = flopy.utils.CellBudgetFile(os.path.join(model_ws, f"{model_name}.cbc"))
frf = cbb.get_data(text='FLOW RIGHT FACE')[0]
fff = cbb.get_data(text='FLOW FRONT FACE')[0]

# Visualize the flow field in the third layer (layer index 2)
pmv = flopy.plot.PlotMapView(model=mf, layer=2)
pmv.plot_vector(frf[2], fff[2])
plt.title("Groundwater flow field in layer 3")
plt.show()

### Courant Number (Co)
This code cell is used to calculate and visualize the Courant number for the third layer of a groundwater model. The Courant number is a dimensionless number that characterizes the stability of numerical solutions in advection-dominated transport problems. It is important for ensuring numerical stability in finite-difference methods used for groundwater modeling.

In [ ]:
# Calculating and plotting the Courant number
hds = flopy.utils.HeadFile(os.path.join(model_ws, f"{model_name}.hds"))
head = hds.get_data(kstpkper=(0, 0))

# Calculating the Darcy velocity as an intermediary step
qx = frf[2] / (delr * delc)  # specific discharge in x-direction
qy = fff[2] / (delr * delc)  # specific discharge in y-direction

# Calculation of the pore water velocity by taking the correct flow area as reference
vx = qx / porosity
vy = qy / porosity

# Calculating the Courant number
dt = perlen[1] / nstp[1]  # time step length
courant_x = np.abs(vx) * dt / delr
courant_y = np.abs(vy) * dt / delc
courant = np.maximum(courant_x, courant_y)

# Plot Courant number
plt.figure(figsize=(10, 8))
pmv = flopy.plot.PlotMapView(model=mf, layer=2)
courant_plot = pmv.plot_array(courant, cmap="jet")
plt.colorbar(courant_plot, shrink=0.7, label="Courant-Zahl")
plt.title("Courant-Zahl in Layer 3")
plt.show()

print(f"Maximal Courant number: {courant.max()}")
print(f"Minimal Courant number: {courant.min()}")
print(f"Average Courant number: {courant.mean()}")

### Peclet Number (Pe)
The Peclet number (Pe) is a dimensionless number used in fluid dynamics to characterize the relative importance of advection and diffusion processes in the transport of solutes. A high Peclet number indicates that advection dominates over diffusion, while a low Peclet number suggests that diffusion is more significant.

In [ ]:
# Calculation of the Peclet number
peclet_x = (np.abs(vx) * delr) / dmcoef
peclet_y = (np.abs(vy) * delc) / dmcoef
peclet = np.maximum(peclet_x, peclet_y)

# Plot Peclet number
plt.figure(figsize=(10, 8))
pmv = flopy.plot.PlotMapView(model=mf, layer=2)
peclet_plot = pmv.plot_array(peclet, cmap="jet")
plt.colorbar(peclet_plot, shrink=0.7, label="Peclet Number")
plt.title("Peclet Number in Layer 3")
plt.show()

print(f"Maximal Peclet number: {peclet.max()}")
print(f"Minimal Peclet number: {peclet.min()}")
print(f"Average Peclet number: {peclet.mean()}")


# Further Resources, literature and tips
- The groundwater modelling part is in large parts adapted from this tutorial: https://www.youtube.com/watch?v=xDgWjArrHNY&t=262s
- Useful explanation about the errors associated with the finite difference approximation: https://www.youtube.com/watch?v=9fGaTU1-f-0&ab_channel=SteveBrunton
- A github page with lots of jupyter notebooks and additional information on groundwater anbd contaminant simulations with Modflow and FloPy can be found here: https://github.com/zahasky/Contaminant-Hydrogeology-Activities/tree/master
- Hatari labs provides a lot of useful tutorials on their webpage and on youtube: https://hatarilabs.com/ih-en/regional-groundwater-modeling-with-modflow-and-flopy-tutorial
- A very informative powerpoint presentation: https://www2.hawaii.edu/~jonghyun/classes/S18/CEE696/files/05_flopy_installation.pdf
- A complete description of the mathematical model, finite differences and all parameters can be found here: https://inside.mines.edu/~epoeter/583CSM/DOC4_MODFLOW2005-TM6A16.pdf. This would enable you to build Moflow from scratch in Python.
- An execelent resource on groundwater modelling with Modflow, but in German language: https://www.schaefer-gwm.de/downloads/mod_gws.pdf
- A comprehensive dictionary about groundwater modelling terminology: https://www.dws.gov.za/Groundwater/Groundwater_Dictionary/index.html?introduction_specific_yield.htm
- Pyvista is an excellent package suitable for groundwater modelling to use in combination with FloPy
- Analytical solutions to the transport equation provide a good plausibility check to your approximate numerical solutions
- Sharing your jupyter notebooks with others is possible with binder: https://www.youtube.com/watch?v=owSGVOov9pQ&ab_channel=SerenaBonaretti. Once you have a repository on Github with a requirements.txt file and your .ipynb notebooks, you can still launch them after a few months and a new container is set up based on the GIT repository. Also pay attention to the binder badge which allows you to execute interactive Github repositories with notebooks in your browser. Binder uses Docker containers and Linux, so you will need to get the linux executables if you want to set up your own repository with FloPy notebooks in binder.
- A script on modelling with MT3DMS: https://inside.mines.edu/~epoeter/583CSM/12-1_MT3D.pdf
- You can open your jupyter notebooks in the browser by installing <jupyter> and <notebook> with pip. You can save it from there as well.